In [1]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_community.document_loaders import JSONLoader
from dotenv import load_dotenv
import os
import pandas as pd
import json
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

load_dotenv()

True

In [80]:
df_KTAS_adult = pd.read_csv('../data/KTAS_성인.csv', encoding='utf-8-sig')
df_KTAS_adult.head(10)

,성인/소아,신체계통,증상,1/2차 구분,KTAS 단계,고려사항
0,성인,심혈관계,심정지(비외상성),2차,1,심정지 (비외상성)
1,성인,심혈관계,심정지(비외상성),2차,5,심폐소생술의 대상이 아닌 경우
2,성인,심혈관계,심정지(외상성),2차,1,심정지 (외상성)
3,성인,심혈관계,심정지(외상성),2차,5,심폐소생술의 대상이 아닌 경우
4,성인,심혈관계,흉통 (심장성),1차,1,중증 호흡곤란
5,성인,심혈관계,흉통 (심장성),1차,1,쇼크
6,성인,심혈관계,흉통 (심장성),1차,1,무의식
7,성인,심혈관계,흉통 (심장성),1차,2,중등도 호흡곤란
8,성인,심혈관계,흉통 (심장성),1차,2,혈역학적 장애
9,성인,심혈관계,흉통 (심장성),1차,2,의식변화


In [ ]:
# JSON 구조로 변환: 증상별로 object 구분

json_KTAS_adult = []
for symptom, group in df_KTAS_adult.groupby('증상'):
    object = {
        "symptom": symptom,
        "ROS": group['신체계통'].values[0],
        "primary": [],
        "secondary": []
    }

    for _, row in group.iterrows():
        type = row['1/2차 구분']
        type = 'primary' if type == '1차' else 'secondary'
        entry = {
            'consideration': row['고려사항'],
            'KTAS_level': row['KTAS 단계']
        }
        object[type].append(entry)

    json_KTAS_adult.append(object)

In [ ]:
json_KTAS_adult

[{'symptom': '20주 미만의 임신',
  'ROS': '임신 / 여성생식계',
  'primary': [{'consideration': '중증 호흡곤란', 'KTAS_level': 1},
   {'consideration': '쇼크', 'KTAS_level': 1},
   {'consideration': '무의식', 'KTAS_level': 1},
   {'consideration': '중등도 호흡곤란', 'KTAS_level': 2},
   {'consideration': '혈역학적 장애', 'KTAS_level': 2},
   {'consideration': '의식변화', 'KTAS_level': 2},
   {'consideration': '열, 면역저하 상태', 'KTAS_level': 2},
   {'consideration': '패혈증 의증', 'KTAS_level': 2},
   {'consideration': '급성 중심성 중증 통증', 'KTAS_level': 2},
   {'consideration': '출혈성 질환 (생명 혹은 사지를 소실할 정도의 위급한 출혈)', 'KTAS_level': 2},
   {'consideration': '고위험성 사고기전', 'KTAS_level': 2},
   {'consideration': '경증 호흡곤란', 'KTAS_level': 3},
   {'consideration': '비정상 맥박수지만 혈역학적으로 안정', 'KTAS_level': 3},
   {'consideration': '전신염증반응증후군', 'KTAS_level': 3},
   {'consideration': '열 (아파 보임)', 'KTAS_level': 3},
   {'consideration': '급성 중심성 중등도 통증', 'KTAS_level': 3},
   {'consideration': '출혈성 질환 (중등도나 경도의 출혈)', 'KTAS_level': 3},
   {'consideration': '급성 중심성 경

In [ ]:
with open('../data/KTAS_성인.json', 'w') as f:
    json.dump(json_KTAS_adult, f)

In [2]:
with open('../data/KTAS_성인.json', 'r') as f:
    json_KTAS_adult = json.load(f)

In [ ]:
# 임베딩으로 위해 JSON을 document로 변환

def metadata_func(record: dict, metadata: dict) -> dict:
    # symptom을 제외한 모든 필드를 metadata로 추가
    for key, value in record.items():
        if key != 'symptom':
            if isinstance(value, dict):
                value.astype(str)
            metadata_dict = {key: value}
            metadata.update(metadata_dict)

    # source와 seq 키가 있다면 명시적으로 제거
    if 'source' in metadata:
        del metadata['source']
    if 'seq_num' in metadata:
        del metadata['seq_num']

    return metadata

loader = JSONLoader(
    file_path='../data/KTAS_성인.json',
    jq_schema=".[]",
    content_key="symptom",
    metadata_func=metadata_func
)

In [85]:
docs = loader.load()
docs[:5]

[Document(metadata={'ROS': '임신 / 여성생식계', 'primary': [{'consideration': '중증 호흡곤란', 'KTAS_level': 1}, {'consideration': '쇼크', 'KTAS_level': 1}, {'consideration': '무의식', 'KTAS_level': 1}, {'consideration': '중등도 호흡곤란', 'KTAS_level': 2}, {'consideration': '혈역학적 장애', 'KTAS_level': 2}, {'consideration': '의식변화', 'KTAS_level': 2}, {'consideration': '열, 면역저하 상태', 'KTAS_level': 2}, {'consideration': '패혈증 의증', 'KTAS_level': 2}, {'consideration': '급성 중심성 중증 통증', 'KTAS_level': 2}, {'consideration': '출혈성 질환 (생명 혹은 사지를 소실할 정도의 위급한 출혈)', 'KTAS_level': 2}, {'consideration': '고위험성 사고기전', 'KTAS_level': 2}, {'consideration': '경증 호흡곤란', 'KTAS_level': 3}, {'consideration': '비정상 맥박수지만 혈역학적으로 안정', 'KTAS_level': 3}, {'consideration': '전신염증반응증후군', 'KTAS_level': 3}, {'consideration': '열 (아파 보임)', 'KTAS_level': 3}, {'consideration': '급성 중심성 중등도 통증', 'KTAS_level': 3}, {'consideration': '출혈성 질환 (중등도나 경도의 출혈)', 'KTAS_level': 3}, {'consideration': '급성 중심성 경증 통증', 'KTAS_level': 4}], 'secondary': [{'consideration': '주기적

In [ ]:
# 임베딩 모델

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task = 'feature-extraction',
    huggingfacehub_api_token=os.environ["HF_API_TOKEN"]
)

In [ ]:
# 임베딩

db = FAISS.from_documents(docs, hf_embeddings, distance_strategy = DistanceStrategy.COSINE)

In [ ]:
# 벡터 DB 저장

db.save_local('db/KTAS_adult_faiss')